<a href="https://colab.research.google.com/github/sumyu199/MATH3026_coursework1_14335252/blob/main/car.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas_datareader as web
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import datetime
from scipy.optimize import minimize
import random
import itertools
import numpy

In [ ]:
def load_data():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    html = pd.read_html(url, header = 0)
    df = html[0]
    return df

#store the loaded data and group the data by sector
df = load_data()

df

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...,...
500,YUM,Yum! Brands Inc,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
501,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
502,ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
503,ZION,Zions Bancorp,reports,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


In [ ]:
#extract data we needed 
topstock = pd.DataFrame()
topstock["Symbol"] = df["Symbol"]
topstock["Security"] = df["Security"]
topstock["GICS Sector"] = df["GICS Sector"]
topstock

,Symbol,Security,GICS Sector
0,MMM,3M Company,Industrials
1,ABT,Abbott Laboratories,Health Care
2,ABBV,AbbVie Inc.,Health Care
3,ABMD,Abiomed,Health Care
4,ACN,Accenture,Information Technology
...,...,...,...
500,YUM,Yum! Brands Inc,Consumer Discretionary
501,ZBRA,Zebra Technologies,Information Technology
502,ZBH,Zimmer Biomet,Health Care
503,ZION,Zions Bancorp,Financials


In [ ]:
#group the sector 
sector = topstock['GICS Sector'].unique().tolist()
sector

['Industrials',
 'Health Care',
 'Information Technology',
 'Communication Services',
 'Consumer Discretionary',
 'Utilities',
 'Financials',
 'Materials',
 'Real Estate',
 'Consumer Staples',
 'Energy']

In [ ]:
#we randomly select 10 stocks from all 11 industries
symbols = []
for i in sector:
  data = topstock.loc[topstock["GICS Sector"] == i ]
  symbols.append(random.sample(data['Symbol'].tolist(), 5))
print(list(zip(sector,symbols)))

[('Industrials', ['IEX', 'EMR', 'LDOS', 'J', 'RSG']), ('Health Care', ['UNH', 'HUM', 'DXCM', 'MRK', 'SYK']), ('Information Technology', ['TER', 'CSCO', 'VRSN', 'INTC', 'AMAT']), ('Communication Services', ['GOOG', 'CHTR', 'T', 'FOX', 'FB']), ('Consumer Discretionary', ['DHI', 'DLTR', 'HD', 'BWA', 'MHK']), ('Utilities', ['AES', 'CMS', 'ES', 'LNT', 'ED']), ('Financials', ['AJG', 'FITB', 'AON', 'MTB', 'MS']), ('Materials', ['PPG', 'ALB', 'MLM', 'AMCR', 'SHW']), ('Real Estate', ['CCI', 'VTR', 'O', 'SPG', 'WELL']), ('Consumer Staples', ['LW', 'CPB', 'STZ', 'KO', 'KHC']), ('Energy', ['NOV', 'PSX', 'DVN', 'APA', 'PXD'])]


In [ ]:
combin_industry = []

combinations_object = itertools.combinations(symbols, 3)
combinations_list = list(combinations_object)
combin_industry.append(combinations_list)



In [ ]:
symbols = combin_industry[0][0]
symbols = list(itertools.chain.from_iterable(symbols))
symbols

['IEX',
 'EMR',
 'LDOS',
 'J',
 'RSG',
 'UNH',
 'HUM',
 'DXCM',
 'MRK',
 'SYK',
 'TER',
 'CSCO',
 'VRSN',
 'INTC',
 'AMAT']

In [ ]:

all_stocks = pd.DataFrame()
start = datetime.datetime(2020, 12, 1)
end = datetime.datetime(2021, 4, 16)
  
for symbol in symbols:
    data = web.DataReader(symbol, 'yahoo', start, end)
    data.to_csv('histdata.csv')
    data = pd.read_csv('histdata.csv')
    tmp = data['Close']
    all_stocks = pd.concat([all_stocks, tmp], axis=1)
 
all_stocks.columns=symbols
  #return including transaction cost 


In [ ]:
all_stocks

In [ ]:
all_stocks.shift(1)

In [ ]:
cov_matrix = all_stocks.pct_change().apply(lambda x: np.log(1+x)).cov()
cov_matrix

In [ ]:
corr_matrix = all_stocks.pct_change().apply(lambda x: np.log(1+x)).corr()
corr_matrix

In [ ]:
returns = np.log(all_stocks/all_stocks.shift(1)).dropna(how="any")

returns

In [ ]:
returns.plot(figsize=(12,10))

In [ ]:
returns.mean()

In [ ]:
returns.mean().mean()

In [ ]:
# the objective function is to minimize the portfolio risk
def objective(weights): 
    weights = np.array(weights)
    return weights.dot(returns.cov()).dot(weights.T)
 
 
 
# The constraints
cons = (# The weights must sum up to one.
        {"type":"eq", "fun": lambda x: np.sum(x)-1}, 
        # This constraints says that the inequalities (ineq) must be non-negative.
        # The expected daily return of our portfolio and we want to be at greater than 0.003
        {"type": "ineq", "fun": lambda x: np.sum(returns.mean()*x)-0.003}) 
 
 
# Every stock can get any weight from 0 to 1
bounds = tuple((0,1) for x in range(returns.shape[1])) 
 
# Initialize the weights with an even split
# In out case each stock will have 10% at the beginning
guess = [1./returns.shape[1] for x in range(returns.shape[1])]
 
 
optimized_results = minimize(objective, guess, method = "SLSQP", bounds=bounds, constraints=cons)
optimized_results

In [ ]:
optimized_results.x

In [ ]:
# we get 1
np.sum(optimized_results.x)

In [ ]:
np.sum(returns.mean()*optimized_results.x)

In [ ]:
port1 = pd.DataFrame(list(zip(symbols, optimized_results.x)), columns=['Symbol', 'Weight'])
port1

In [ ]:
port1_dict = dict(zip(symbols, optimized_results.x))

In [ ]:
port_var1 = cov_matrix.mul(port1_dict, axis=0).mul(port1_dict, axis=1).sum().sum()
port_var1

In [ ]:
returns1 = returns*0.975

In [ ]:
returns1.mean()

In [ ]:
returns1.mean().mean()

In [ ]:
# the objective function is to minimize the portfolio risk
def objective2(weights): 
    weights = np.array(weights)
    return weights.dot(returns1.cov()).dot(weights.T)
 
 
 
# The constraints
cons2 = (# The weights must sum up to one.
        {"type":"eq", "fun": lambda x: np.sum(x)-1}, 
        # This constraints says that the inequalities (ineq) must be non-negative.
        # The expected daily return of our portfolio and we want to be at greater than 0.003
        {"type": "ineq", "fun": lambda x: np.sum(returns1.mean()*x)-0.003}) 
 
 
# Every stock can get any weight from 0 to 1
bounds2 = tuple((0,1) for x in range(returns1.shape[1])) 
 
# Initialize the weights with an even split
# In out case each stock will have 10% at the beginning
guess2 = [1./returns1.shape[1] for x in range(returns1.shape[1])]
 
 
optimized_results2 = minimize(objective2, guess2, method = "SLSQP", bounds=bounds2, constraints=cons2)
optimized_results2

In [ ]:
optimized_results2.x

In [ ]:
np.sum(optimized_results2.x)

In [ ]:
np.sum(returns1.mean()*optimized_results2.x)

In [ ]:
pd.DataFrame(list(zip(symbols, optimized_results2.x)), columns=['Symbol', 'Weight'])

In [ ]:
port2_dict = dict(zip(symbols, optimized_results2.x))

In [ ]:
port_var2 = cov_matrix.mul(port2_dict, axis=0).mul(port2_dict, axis=1).sum().sum()
port_var2

In [ ]:
ann_sd = all_stocks.pct_change().apply(lambda x: np.log(1+x)).std().apply(lambda x: x*np.sqrt(94))
ann_sd